In [1]:
import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from smart_open import smart_open
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from nltk.corpus import stopwords
import os, codecs
%matplotlib inline
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from sklearn import metrics as skmetrics
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

2017-08-27 11:20:24,405 : INFO : 'pattern' package found; tag filters are available for English
/usr/lib64/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Dictionary features

In [2]:
personalities = pd.read_csv('~/personality-normalized-word2vec-norm.csv', encoding='utf-8')

In [3]:
personalities.shape

(1039, 186)

In [4]:
personalities = personalities.convert_objects(convert_numeric=True)

/usr/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


In [5]:
personalities.agreeabeness_m.value_counts()

1    542
0    497
Name: agreeabeness_m, dtype: int64

In [6]:
pre_proc_attr = ['skip', 'compound', 'hashtags', 'links', 'punct', 'questions', 'exclam', 'numbers', 'upcase', 'lowcase', 'firstup', 'pt.lexicon', 'added', 'verb.pro', 'names', 'en.lexicon', 'rewrite', 'mispell', 'foreign', 'laugh', 'emph', 'echars', 'unkn']

In [7]:
grammar_attr = ['m', 'f', 's', 'p', 'aument', 'dimin', 'superlat', 'N', 'A', 'PREP', 'CONJ', 'ADV', 'PREFIX', 'SIGLA', 'ABREV', 'INTERJ', 'DET', 'def', 'indef', 'NUM', 'numC', 'numO', 'numM', 'numF', 'PRO', 'proDem', 'proIndef', 'proRel', 'proInterr', 'proTrat', 'proPoss', 'proPess', 'acusativa', 'dativa', 'nominativa', 'obliqua', 'reflexa', 'p1', 'p2', 'p3', 'V', 'VW', 'VG', 'VK', 'VP', 'VI', 'VJ', 'VF', 'VQ', 'VS', 'VT', 'VU', 'VY', 'VC', 'V1s', 'V2s', 'V3s', 'V1p', 'V2p', 'V3p']

In [8]:
liwc_attr = ['X1funct' ,'X2pronoun' ,'X3ppron','X4i','X5we','X6you','X7shehe','X8they','X9ipron','X10article','X11verb','X12auxverb','X13past','X14present','X15future','X16adverb','X17preps','X18conj','X19negate','X20quant','X21number','X22swear','X23social','X24family','X25friend','X26humans','X27affect','X28posemo','X29negemo','X30anx','X31anger','X32sad','X33cogmech','X34insight','X35cause','X36discrep','X37tentat','X38certain','X39inhib','X40incl','X41excl','X42percept','X43see','X44hear','X45feel','X46bio','X47body','X48health','X49sexual','X50ingest','X51relativ','X52motion','X53space','X54time','X55work','X56achieve','X57leisure','X58home','X59money','X60relig','X61death','X62assent','X63nonfl', 'X64filler']

In [9]:
all_attr = personalities[7:157]

In [10]:
def remove_nan(atributes):
    personalities_attributes = np.array(personalities[atributes])
    personalities_attributes[np.isnan(personalities_attributes)] = 0
    return personalities_attributes

In [11]:
data = remove_nan(liwc_attr)

In [12]:
clf = RandomForestClassifier(max_depth=2, random_state=42, criterion= "gini", class_weight="balanced", n_estimators=1000)

## Extraversion

In [13]:
scores = cross_val_score(clf, data, personalities['extraversion_ober_2'], cv=10, scoring='f1_macro')

In [14]:
print("F1-score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

F1-score: 0.29 (+/- 0.07)


## Agreableness

In [15]:
scores = cross_val_score(clf, data, personalities['agreeableness_ober_2'], cv=10, scoring='f1_macro')

In [16]:
scores

array([ 0.22519437,  0.22833599,  0.27440914,  0.23400509,  0.24840856,
        0.28932423,  0.1811828 ,  0.24034059,  0.21476813,  0.20727963])

In [17]:
print("F1-score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

F1-score: 0.23 (+/- 0.06)


## Conscientiousness

In [18]:
scores = cross_val_score(clf, data, personalities['conscientiousness_ober_2'], cv=10, scoring='f1_macro')

In [19]:
print("F1-score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

F1-score: 0.27 (+/- 0.08)


## Neuroticism

In [20]:
scores = cross_val_score(clf, data, personalities['neuroticism_ober_2'], cv=10, scoring='f1_macro')

In [21]:
print("F1-score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

F1-score: 0.31 (+/- 0.09)


## Openness

In [22]:
scores = cross_val_score(clf, data, personalities['openness_ober_2'], cv=10, scoring='f1_macro')

In [23]:
print("F1-score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

F1-score: 0.29 (+/- 0.10)
